In [13]:
# Import packages used in this script
import numpy as np
import pandas as pd
import random
import time
import os
import keras

## Data, can also load images from other places
from keras.datasets import cifar10

## Build model, load model
from keras.models import Sequential, load_model

## Dense -> Fully Connected
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

## Activation functions
from keras.layers import Dropout, LeakyReLU

## Optimizers
from keras.optimizers import RMSprop, Adam

## Callbacks --- Save Model (one of its parameters)
from keras.callbacks import ModelCheckpoint

## Split whole dataset into two groups(training / validation set)
from sklearn.model_selection import train_test_split

## Compute the accuracy score for the model
from sklearn.metrics import accuracy_score

## Encode integer label into vector
from sklearn.preprocessing import OneHotEncoder

## Plot function
import matplotlib.pyplot as plt

## Data augmentation
from keras.preprocessing.image import ImageDataGenerator

## Load mnist dataset, confirm the shape and the contents of the data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print("Shape of the training set: {}".format(x_train.shape))
print("Shape of the testing set: {}".format(x_test.shape))
print("Ground truth label (training set): {}".format(y_train))
print("Ground truth label (testing set): {}".format(y_test))

## Normalize data in the range(0 ~ 1)
x_train = x_train / 255
x_test = x_test / 255

## Encode integer label into one-hot vector
## e.g. 3 -> [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
## e.g. 5 -> [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
y_train = keras.utils.to_categorical(y_train.reshape(-1), 10)
y_test = keras.utils.to_categorical(y_test.reshape(-1), 10)
print("Shape of the One-hot label: {}, {}".format(y_train.shape, y_test.shape))

# Modified AlexNet
model = Sequential()  ## build model graph

## Add the convolution layer (named conv_1) to the graph
## Filter size = 11 * 11, channel = 96, activation function = relu
## Weight initializer following normal distribution
model.add(Conv2D(96, kernel_size = (11, 11), 
                 strides = (4, 4),
                 input_shape = (32, 32, 3),
                 padding = 'same', activation = 'relu',
                 kernel_initializer = 'he_normal', name = 'conv_1'))

## Add the pooling layer (named pooling_1) to the graph
## Filter size = 3* 3, stride = 2
## Pad zero to the feature map boundary
model.add(MaxPooling2D(pool_size = (3, 3), strides = (2, 2),
                       padding = 'same', name = 'pooling_1'))

## Add the convolution layer (named conv_2) to the graph
## Filter size = 5 * 5, channel = 256, activation function = relu
## Weight initializer following normal distribution
model.add(Conv2D(256, kernel_size = (5, 5),
                 strides = (1, 1),
                 padding = 'same', activation = 'relu',
                 kernel_initializer = 'he_normal', name = 'conv_2'))

## Add the pooling layer (named pooling_2) to the graph
## Filter size = 3* 3, stride = 2
## Pad zero to the feature map boundary
model.add(MaxPooling2D(pool_size = (3, 3), strides = (2, 2),
                       padding = 'same', name = 'pooling_2'))

## Add the convolution layer (named conv_3) to the graph
## Filter size = 3* 3, channel = 384, activation function = relu
## Weight initializer following normal distribution
model.add(Conv2D(384, kernel_size = (3, 3),
                 strides = (1, 1),
                 padding = 'same', activation = 'relu',
                 kernel_initializer = 'he_normal', name = 'conv_3'))

## Add the convolution layer (named conv_4) to the graph
## Filter size = 3* 3, channel = 384, activation function = relu
## Weight initializer following normal distribution
model.add(Conv2D(384, kernel_size = (3, 3),
                 strides = (1, 1),
                 padding = 'same', activation = 'relu',
                 kernel_initializer = 'he_normal', name = 'conv_4'))

## Add the convolution layer (named conv_5) to the graph
## Filter size = 3* 3, channel = 384, activation function = relu
## Weight initializer following normal distribution
model.add(Conv2D(256, kernel_size = (3, 3),
                 strides = (1, 1),
                 padding = 'same', activation = 'relu',
                 kernel_initializer = 'he_normal', name = 'conv_5'))

## Add the pooling layer (named pooling_3) to the graph
## Filter size = 3 * 3, stride = 2
## Pad zero to the feature map boundary
model.add(MaxPooling2D(pool_size = (3, 3), strides = (2, 2),
                       padding = 'same', name = 'pooling_3'))

## Reshape the feature map into vectors
model.add(Flatten(name = 'flatten'))

## Add the fully connected layer (named fs_1) to the graph
## Units = 512, activation function = relu
## Weight initializer following normal distribution
## Weight shape = (256, 512)
model.add(Dense(512, activation = 'relu',
                kernel_initializer = 'he_normal', name = 'fs_1'))

model.add(Dropout(0.8))

## Add the fully connected layer (named fs_2) to the graph
## Units = 256, activation function = relu
## Weight initializer following normal distribution
## Weight shape = (512, 256)
model.add(Dense(256, activation = 'relu',
                kernel_initializer = 'he_normal', name = 'fs_2'))

## Add the output layer (named logit) to the graph
## Units = 10, activation function = softmax (normalize)
## Weight initializer following normal distribution
## Weight shape = (256, 10)
model.add(Dense(units = 10, activation = 'softmax',
                kernel_initializer = 'he_normal', name = 'logit'))

## Define the objection function and optimizer
## Define the evaluation metrics: accuracy (for classification)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',
              metrics = ['accuracy'])

## Print the detail of the model
model.summary()

'''
## Training with data augmentation
## *******************************
datagen = ImageDataGenerator(rotation_range = 20,
                            width_shift_range = 0.2,
                            height_shift_range = 0.2,
                            zoom_range = 0.2,
                            shear_range = 0.2)
datagen.fit(x_train)
s = datagen.flow(x_train, y_train, batch_size = 256)
history = model.fit_generator(s, steps_per_epoch = 100, epochs = 20,
                             validation_data = (x_test, y_test),
                             callbacks = [ModelCheckpoint('CIFAR10_sample_model.h5',
                                                         monitor = 'val_accuracy',
                                                         save_best_only = True)])

### Train the whole model using "fit" function
### Epoch: Number of training loops
### Shuffle: perturb order of the training data
### Save the model (named Cifar10_sample_model.h5)
### Save the model only when the testing accuracy achieves the best
history = model.fit(x_train, y_train, epochs = 20, 
                    validation_data = (x_test, y_test),
                    shuffle = True, batch_size = 256, 
                    callbacks = [ModelCheckpoint('CIFAR10_sample_model.h5',
                                                 monitor = 'val_accuracy',
                                                 save_best_only = True)])
'''
model_test = load_model('CIFAR10_sample_model.h5') 

### Plot testing data
def plot(a):
    '''
    Multiply 255 to the normalized data
    and transform the data type from float into int
    '''
    a *= 255
    a = a.astype(np.uint8)
    return a.reshape(32, 32, 3)

# ********** Display testing images **********
#print("n = ", end = '')
#n = int(input())
#ax = plt.subplot(1, 1, 1)
#plt.imshow(plot(x_test[n]), cmap = 'gray')
#ax.get_xaxis().set_visible(False)
#ax.get_yaxis().set_visible(False)
    
# Build category dictionary
category = {
    0: 'airplane',
    1: 'automobile',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck'
}
    
### Print inference results corresponding to the images we plot
#prediction = model_test.predict(x_test[n].reshape(-1, 32, 32, 3))
#print("Inference label: {}".format(category[np.argmax(prediction, axis = 1)[0]]))
#print("Ground truth label: {}".format(category[np.argmax(y_test[n], axis = 0)])) 
    # y_test[n] is one-hot-encoded, use np.argmax to decode
    
pred = model_test.predict(x_test.reshape(-1, 32, 32, 3))
predd = np.argmax(pred, axis = 1)
ans = np.argmax(y_test, axis = 1)
sum = 0
for i in range(10000):
    if predd[i] == ans[i]:
        sum += 1
accuracy = sum / 10000
print("Validation accuracy: ")
print(accuracy)
    
# parameters: 4,012,682

Shape of the training set: (50000, 32, 32, 3)
Shape of the testing set: (10000, 32, 32, 3)
Ground truth label (training set): [[6]
 [9]
 [9]
 ...
 [9]
 [1]
 [1]]
Ground truth label (testing set): [[3]
 [8]
 [8]
 ...
 [5]
 [1]
 [7]]
Shape of the One-hot label: (50000, 10), (10000, 10)
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 8, 8, 96)          34944     
_________________________________________________________________
pooling_1 (MaxPooling2D)     (None, 4, 4, 96)          0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 4, 4, 256)         614656    
_________________________________________________________________
pooling_2 (MaxPooling2D)     (None, 2, 2, 256)         0         
_________________________________________________________________
conv_3 (Conv2D)              (No